In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()
dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]
erAppId = os.environ["APP_ID"]

DATABASE_CONNECTION_URI = os.environ["NEW_DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

class OdooAPI:
    def __init__(self):
        self.dbUrl = dbUrl
        self.dbName = dbName
        self.dbUsername = dbUsername
        self.dbPassword = dbPassword
        self.common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(self.dbUrl))
        self.uid = self.common.authenticate(self.dbName, self.dbUsername, self.dbPassword, {})
        self.models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(self.dbUrl))
        # self.country_codes = {
        #     "Argentina": "AR",
        #     "Paraguay": "PY",
        #     "Bolivia": "BO",
        #     "Uruguay": "UY",
        #     "Chile": "CL",
        #     "Perú": "PE",
        #     "Peru": "PE",
        #     "Ecuador": "EC",
        #     "Colombia": "CO",
        #     "Venezuela": "VE",
        #     "Brasil": "BR",
        #     "Panamá": "PA",
        #     "Panama": "PA",
        #     "Costa Rica": "CR",
        #     "Honduras": "HN",
        #     "El Salvador": "SV",
        #     "Guatemala": "GT",
        #     "México": "MX",
        #     "Mexico": "MX",
        #     "República Dominicana": "DM",
        #     "Republica Dominicana": "DM",
        #     "Rep Dom": "DM",
        #     "Puerto Rico": "PR",
        #     "España": "ES",
        # }

    # # Data Handling Functions
    # def id_field_into_1value(self, jsonFile):
    #     for index, line in enumerate(jsonFile):
    #         for item in line:
    #             if type(line[item]) == list:
    #                 jsonFile[index][item] = jsonFile[index][item][1]

    def raw_export_relation(self, properties_dict):
        sale_orders = self.models.execute_kw(self.dbName,
                                               self.uid,
                                               self.dbPassword,
                                               'sale.order',
                                               'search_read',
                                               [],
                                               {'fields': list(properties_dict.keys())})
        new_data = []
        #print(len(sale_orders))
        for sale_order in sale_orders:
            order_name = sale_order["name"]
            order_line_ids = sale_order["order_line"]  # Even if it's empty, it will still return []
            
            if len(order_line_ids) > 0:
                for order_line_id in order_line_ids:
                    new_data.append({"order_name": order_name, "order_line_id": order_line_id})
            else:
                # If no order lines associated, create a row with a placeholder for order line ID
                new_data.append({"order_name": order_name, "order_line_id": None})

        # Convert the new data into a DataFrame
        new_df = pd.DataFrame(new_data)
        new_df.to_sql('odoo_sale_orders_to_sale_lines', con=engine, if_exists='replace', index=False)
        

        # # Use list comprehension to iterate over each dictionary and remove the "order_line" key
        # data_without_order_line = [{k: v for k, v in item.items() if k != 'order_line'} for item in subscriptions]
        # #print(data_without_order_line)
        # self.id_field_into_1value(data_without_order_line)
        # df = pd.DataFrame(data_without_order_line)
        # # insert the dataframe into the postgres table
        # df.to_sql('odoo_subscriptions_test', con=engine, if_exists='replace', index=False)
        # #df.to_csv("raw_odoo_subscriptions.csv", encoding="latin-1", index=False, errors='ignore')

o = OdooAPI()

properties_dict = {
  "create_date": 1,
  "name": 1,
  "order_line": 1
}

o.raw_export_relation(properties_dict)

